# Configurações iniciais e extraindo dados do bucket.

In [ ]:
import requests
import json
import pymysql
import pandas as pd

# Inserir as credenciais do banco de dados, menos o nome do banco de dados
db_config = {
    "host": "",
    "user": "",
    "password": "",
    "database": "cb_labs_db",
}



public_url = "https://storage.googleapis.com/cblabs_datalake/bronze/res_guest_checks/2024/07/24/ERP.json"

response = requests.get(public_url)

if response.status_code == 200:
    data = json.loads(response.text)
    print("Dados carregados com sucesso!")
    print(data)
else:
    print(f"Erro ao acessar o arquivo: {response.status_code}")
    data = None



Dados carregados com sucesso!
{'curUTC': '2024-05-05T06:06:06', 'locRef': '99 CB CB', 'guestChecks': [{'guestCheckId': 1122334455, 'chkNum': 1234, 'opnBusDt': '2024-01-01', 'opnUTC': '2024-01-01T09:09:09', 'opnLcl': '2024-01-01T06:09:09', 'clsdBusDt': '2024-01-01', 'clsdUTC': '2024-01-01T12:12:12', 'clsdLcl': '2024-01-01T09:12:12', 'lastTransUTC': '2024-01-01T12:12:12', 'lastTransLcl': '2024-01-01T09:12:12', 'lastUpdatedUTC': '2024-01-01T13:13:13', 'lastUpdatedLcl': '2024-01-01T10:13:13', 'clsdFlag': True, 'gstCnt': 1, 'subTtl': 109.9, 'nonTxblSlsTtl': None, 'chkTtl': 109.9, 'dscTtl': -10, 'payTtl': 109.9, 'balDueTtl': None, 'rvcNum': 101, 'otNum': 1, 'ocNum': None, 'tblNum': 1, 'tblName': '90', 'empNum': 55555, 'numSrvcRd': 3, 'numChkPrntd': 2, 'taxes': [{'taxNum': 28, 'txblSlsTtl': 119.9, 'taxCollTtl': 20.81, 'taxRate': 21, 'type': 3}], 'detailLines': [{'guestCheckLineItemId': 9988776655, 'rvcNum': 123, 'dtlOtNum': 1, 'dtlOcNum': None, 'lineNum': 1, 'dtlId': 1, 'detailUTC': '2024-01-

# Inserindo dados no MySQL.

In [16]:
if data:
    conn = pymysql.connect(**db_config)
    cursor = conn.cursor()

    # Inserindo location
    loc_ref = data['locRef']
    try:
        cursor.execute(
            "INSERT IGNORE INTO locations (loc_ref, loc_name) VALUES (%s, %s)",
            (loc_ref, loc_ref)
        )
    except Exception as e:
        print("Erro ao inserir location:", e)

    # Inserindo guest_checks 
    for gc in data['guestChecks']:
        try:
            cursor.execute("""
                INSERT IGNORE INTO guest_checks (
                    guest_check_id, loc_ref, chk_num, opn_bus_dt, opn_utc, opn_lcl, clsd_bus_dt, clsd_utc, clsd_lcl,
                    last_trans_utc, last_trans_lcl, last_updated_utc, last_updated_lcl, clsd_flag, gst_cnt, sub_ttl,
                    non_txbl_sls_ttl, chk_ttl, dsc_ttl, pay_ttl, bal_due_ttl, rvc_num, ot_num, oc_num, tbl_num,
                    tbl_name, emp_num, num_srvc_rd, num_chk_prntd
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, (
                gc['guestCheckId'], loc_ref, gc['chkNum'], gc['opnBusDt'], gc['opnUTC'], gc['opnLcl'],
                gc['clsdBusDt'], gc['clsdUTC'], gc['clsdLcl'], gc['lastTransUTC'], gc['lastTransLcl'],
                gc['lastUpdatedUTC'], gc['lastUpdatedLcl'], gc['clsdFlag'], gc['gstCnt'], gc['subTtl'],
                gc['nonTxblSlsTtl'], gc['chkTtl'], gc['dscTtl'], gc['payTtl'], gc['balDueTtl'], gc['rvcNum'],
                gc['otNum'], gc['ocNum'], gc['tblNum'], gc['tblName'], gc['empNum'], gc['numSrvcRd'], gc['numChkPrntd']
            ))
        except Exception as e:
            print("Erro ao inserir guest_check:", e)

        # Inserindo taxes
        for tax in gc.get('taxes', []):
            try:
                cursor.execute("""
                    INSERT INTO taxes (
                        guest_check_id, tax_num, txbl_sls_ttl, tax_coll_ttl, tax_rate, type
                    ) VALUES (%s, %s, %s, %s, %s, %s)
                """, (
                    gc['guestCheckId'], tax['taxNum'], tax['txblSlsTtl'], tax['taxCollTtl'], tax['taxRate'], tax['type']
                ))
            except Exception as e:
                print("Erro ao inserir tax:", e)

        # Inserindo detail_lines e menu_items
        for dl in gc.get('detailLines', []):
            try:
                cursor.execute("""
                    INSERT INTO detail_lines (
                        guest_check_line_item_id, guest_check_id, line_item_type, dsp_ttl, seat_num, rvc_num, dtl_ot_num,
                        dtl_oc_num, line_num, dtl_id, detail_utc, detail_lcl, last_update_utc, last_update_lcl,
                        bus_dt, ws_num, agg_ttl, agg_qty, svc_rnd_num, chk_emp_id, chk_emp_num, dsp_qty
                    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """, (
                    dl['guestCheckLineItemId'], gc['guestCheckId'], 'menu_item', dl['dspTtl'], dl['seatNum'], dl['rvcNum'],
                    dl['dtlOtNum'], dl['dtlOcNum'], dl['lineNum'], dl['dtlId'], dl['detailUTC'], dl['detailLcl'],
                    dl['lastUpdateUTC'], dl['lastUpdateLcl'], dl['busDt'], dl['wsNum'], dl['aggTtl'], dl['aggQty'],
                    dl['svcRndNum'], dl['chkEmpId'], dl['chkEmpNum'], dl['dspQty']
                ))
                # Inserindo menu_items
                menu = dl.get('menuItem')
                if menu:
                    cursor.execute("""
                        INSERT INTO menu_items (
                            guest_check_line_item_id, mi_num, mod_flag, incl_tax, active_taxes, prc_lvl
                        ) VALUES (%s, %s, %s, %s, %s, %s)
                    """, (
                        dl['guestCheckLineItemId'], menu['miNum'], menu['modFlag'], menu['inclTax'], menu['activeTaxes'], menu['prcLvl']
                    ))
            except Exception as e:
                print("Erro ao inserir detail_line/menu_item:", e)

    conn.commit()
    cursor.close()
    conn.close()
    print("Dados inseridos com sucesso no MySQL!")

Dados inseridos com sucesso no MySQL!


# Mostrando dados dentro do banco.

In [ ]:
conn = pymysql.connect(**db_config)

df = pd.read_sql(f"SELECT * FROM guest_checks", conn)
display(df)

conn.close()

,guest_check_id,loc_ref,chk_num,opn_bus_dt,opn_utc,opn_lcl,clsd_bus_dt,clsd_utc,clsd_lcl,last_trans_utc,...,pay_ttl,bal_due_ttl,rvc_num,ot_num,oc_num,tbl_num,tbl_name,emp_num,num_srvc_rd,num_chk_prntd
0,1122334455,99 CB CB,1234,2024-01-01,2024-01-01 09:09:09,2024-01-01 06:09:09,2024-01-01,2024-01-01 12:12:12,2024-01-01 09:12:12,2024-01-01 12:12:12,...,109.9,None,101,1,None,1,90,55555,3,2
